<p>
50.007 MACHINE LEARNING<br />
2024 SUMMER<br />
<b>COGITO</b><br />
</p>

# Setup

In [ ]:
from base import *
from grid_search import cv_grid_search
from random_search import cv_random_search
from simulated_annealing import SimulatedAnnealing, cv_simulated_annealing

logging.basicConfig(filename="log", level=logging.INFO, format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M")

# Task 1

In [ ]:
def σ(z): return 1 / (1 + np.exp(-z))
# def ce_loss(y, ŷ): return (-1/(y.shape[0])) * np.sum(y * np.log(ŷ) + (1 - y) * np.log(1 - ŷ))

# Return dw and db, for some X, y, ŷ, w, R, and λ.
def gradients_logreg(X, y, ŷ, w, R=None, λ=0):
	m, _ = X.shape
	dw = 1/m * np.dot(X.T, (ŷ - y))
	db = 1/m * np.sum(ŷ - y)
	if R == "L2":
		dw += λ * w / m
	elif R == "L1":
		dw += λ * np.sign(w) / m
	return dw, db

# Return (w, b) from gradient descent on X_train and y_train, for some τ, α, G, β, R, and λ.
def train_logreg(X_train, y_train, τ=1000, α=0.1, G="mini-batch", β=128, R=None, λ=0):
	m, n = X_train.shape
	w, b = np.zeros((n, 1)), 0
	for epoch in range(τ):
		if G == "full-batch":
			X_batch, y_batch = X_train, y_train
			ŷ = σ(np.dot(X_batch, w) + b)
			dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
			w, b = w - α*dw, b - α*db
		elif G == "mini-batch":
			for i in range(0, m, β):
				X_batch, y_batch = X_train[i:i+β], y_train[i:i+β]
				ŷ = σ(np.dot(X_batch, w) + b)
				dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
				w, b = w - α*dw, b - α*db
		elif G == "stochastic":
			for i in range(m):
				X_batch, y_batch = X_train[i:i+1], y_train[i:i+1]
				ŷ = σ(np.dot(X_batch, w) + b)
				dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
				w, b = w - α*dw, b - α*db
	return w, b

# Return array of predictions, where each prediction is 1 if corresponding ŷ entry > 0.5, and 0 otherwise.
def predict_logreg(wb_tuple, X):
	w, b = wb_tuple
	ŷ = σ(np.dot(X, w) + b)
	return np.array([1 if p > 0.5 else 0 for p in ŷ])

# Train model, make predictions, and save predictions to CSV file.
def generate_predictions_logreg(τ=1000, α=0.1, G="mini-batch", β=128, R=None, λ=0):
	start_time = time.time()
	w, b = train_logreg(np.array(X_train), np.array(y_train), τ, α, G, β, R, λ)
	predictions = predict_logreg((w, b), np.array(X_test))
	output_dir = "./predictions/logreg/"
	os.makedirs(output_dir, exist_ok=True)
	file_name = os.path.join(output_dir, f"predictions.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

In [ ]:
# generate_predictions_logreg(τ=1, α=0.08, G="mini-batch", β=128, R=None, λ=0)
# generate_predictions_logreg(τ=1000, α=0.08, G="stochastic", R=L2, λ=0.5)

## Task 2

In [ ]:
def apply_pca(x):
	scaler = StandardScaler()
	X_train_scaled = scaler.fit_transform(X_train)
	X_test_scaled = scaler.transform(X_test)
	if 0 <= x <= 1:
		# x is variance threshold.
		pca = PCA(n_components=None)
		pca.fit(X_train_scaled)
		c = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= x) + 1
		v = x
	else:
		# x is number of components.
		pca = PCA(n_components=x)
		pca.fit(X_train_scaled)
		c = x
		v = sum(pca.explained_variance_ratio_)
	# Transform train and test datasets.
	X_train_pca = pca.transform(X_train_scaled)
	X_test_pca = pca.transform(X_test_scaled)
	return X_train_pca, X_test_pca, c, v

def train_SKLknn(X_train, y_train, k=5):
	model = KNeighborsClassifier(n_neighbors=k)
	model.fit(X_train, y_train)
	return model

def predict_SKLknn(model, X): return model.predict(X)

# Train model, make model predictions, and save model predictions to CSV file.
def generate_predictions_pcaknn(x):
	start_time = time.time()
	X_train_pca, X_test_pca, c, v = apply_pca(x)
	model = train_SKLknn(X_train_pca, y_train, k=2)
	predictions = predict_SKLknn(model, X_test_pca)
	output_dir = "./predictions/pcaknn/"
	os.makedirs(output_dir, exist_ok=True)
	file_name = os.path.join(output_dir, f"predictions.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

In [ ]:
# generate_predictions_pcaknn(100)
# generate_predictions_pcaknn(500)
# generate_predictions_pcaknn(1000)
# generate_predictions_pcaknn(2000)
# generate_predictions_pcaknn(5000)

## Task 3

## SKLet

In [ ]:
# Default:
# generate_predictions(
# 	n_estimators=100,
# 	criterion="gini",
# 	max_depth=None,
# 	min_samples_split=2,
# 	min_samples_leaf=1,
# 	min_weight_fraction_leaf=0.0,
# 	max_features="sqrt",
# 	max_leaf_nodes=None,
# 	min_impurity_decrease=0.0,
# 	bootstrap=False,
# 	random_state=None,
# 	class_weight=None,
# 	ccp_alpha=0.0,
# 	max_samples=None
# )

# generate_predictions(
# 	"SKLet",
# 	n_estimators=150,
# 	criterion="entropy",
# 	max_depth=600,
# 	min_samples_split=160,
# 	min_weight_fraction_leaf=1e-05,
# 	max_features=0.00502,
# 	min_impurity_decrease=1e-05,
# 	bootstrap=True,
# 	class_weight="balanced_subsample",
# 	ccp_alpha=1e-05,
# 	max_samples=0.9
# )

# cv_grid_search("SKLet", grid={
# 	"n_estimators": [150],
# 	"criterion": ["gini"],
# 	"max_depth": [580],
# 	"min_samples_split": [160],
# 	"min_weight_fraction_leaf": [0.00001],
# 	"min_impurity_decrease": [0.00001],
# 	"bootstrap": [True],
# 	"ccp_alpha": [0.00001],
# 	"max_samples": [0.9]
# }, k=5)

## SKLlsvm

In [ ]:
# Default:
# generate_predictions(
# 	"SKLlsvm",
# 	penalty="l2",
# 	loss="squared_hinge",
# 	tol=0.0001,
# 	dual="auto",
# 	C=1.0,
# 	random_state=None,
# 	max_iter=1000
# )

# generate_predictions(
# 	"SKLlsvm",
# 	penalty="l2",
# 	loss="squared_hinge",
# 	tol=0.0001,
# 	dual="auto",
# 	C=0.5,
# 	random_state=None,
# 	max_iter=1200
# )

# cv_grid_search("SKLlsvm", grid={
# 	"penalty": ["l1"],
# 	"loss": ["squared_hinge"],
# 	"dual": [False],
# 	"C": [0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94],
# 	"max_iter": [1200]
# }, k=5)

## SKLmnb

In [ ]:
# Default:
# generate_predictions(
# 	"SKLmnb",
# 	alpha=1.0,
# 	fit_prior=True
# )

# cv_grid_search("SKLmnb", grid={
# 	"alpha": [1.375, 1.376, 1.377, 1.378, 1.379, 1.38, 1.381, 1.382, 1.383, 1.384, 1.385],
# 	"fit_prior": [False]
# }, k=5)

## SKLlogreg

In [ ]:
# generate_predictions(
# 	"SKLlogreg",
# 	C=5.1,
# 	max_iter=300
# )

# cv_grid_search("SKLlogreg", grid={
# 	"C": [0.632, 0.64],
# 	"class_weight": ["balanced"]
# }, k=10)

## SKLhgb

In [ ]:
# Default:
# generate_predictions(
# 	"SKLhgb",
# 	learning_rate=0.1,
# 	max_iter=100,
# 	max_leaf_nodes=31,
# 	max_depth=None,
# 	min_samples_leaf=20,
# 	l2_regularization=0.0,
# 	random_state=None
# )

# cv_grid_search("SKLhgb", grid={
# 	"learning_rate": [0.08],
# 	"max_depth": [80]
# 	"max_iter": [500],
# 	"max_leaf_nodes": [70, 75, 80],
# 	"min_samples_leaf": [15, 18, 20]
# }, k=5)

## XGBgb

In [ ]:
# Default:
# generate_predictions(
# 	"XGBgb",
# 	n_estimators=100,
# 	max_depth=6,
# 	learning_rate=0.3,
# 	booster="gbtree",
# 	subsample=1.0,
# 	reg_alpha=0,
# 	reg_lambda=1,
# 	random_state=None
# )

# cv_grid_search("XGBgb", grid={
# 	"n_estimators": [600, 800, 1000],
# 	"max_depth": [8, 12, 16],
# 	"learning_rate": [0.1],
# 	"subsample": [0.8],
# 	"reg_alpha": [0.1],
# 	"reg_lambda": [2, 2.4]
# }, k=5)

## CBgb

In [9]:
# Default:
# generate_predictions(
# 	"CBgb",
# 	iterations=None,
# 	learning_rate=None,
# 	depth=None,
# 	l2_leaf_reg=None,
#     rsm=None,
# 	random_seed=None,
#     auto_class_weights=None,
# 	random_strength=None
# 	bootstrap_type=None,
# 	subsample=None
# )

# generate_predictions(
# 	"CBgb",
# 	learning_rate=0.1,
# 	depth=8,
# 	l2_leaf_reg=4,
# 	rsm=0.8,
# 	auto_class_weights="Balanced",
# 	random_strength=None,
# 	bootstrap_type=None,
# 	subsample=None
# )

# cv_grid_search("CBgb", grid={
# 	"learning_rate": [0.1],
# 	"max_depth": [12, 14],
# 	"n_estimators": [800],
# 	"reg_lambda": [3.6]
# }, k=5)

# cv_random_search("CBgb", grid={
# 	"learning_rate": uniform(0.001, 0.1),
# 	"max_depth": randint(1, 2),
# 	"n_estimators": randint(1, 2),
# }, k=2, n_iter=1)

# cv_simulated_annealing(model_name="CBgb", grid={
# 	"learning_rate": (0.06, 0.12),
# 	"max_depth": [12],
# 	"reg_lambda": (0.1, 1)
# })

0:	learn: 0.6854800	total: 124ms	remaining: 2m 4s
1:	learn: 0.6782099	total: 243ms	remaining: 2m 1s
2:	learn: 0.6723648	total: 366ms	remaining: 2m 1s
3:	learn: 0.6687896	total: 488ms	remaining: 2m 1s
4:	learn: 0.6652581	total: 609ms	remaining: 2m 1s
5:	learn: 0.6616949	total: 726ms	remaining: 2m
6:	learn: 0.6592428	total: 845ms	remaining: 1m 59s
7:	learn: 0.6565271	total: 962ms	remaining: 1m 59s
8:	learn: 0.6536072	total: 1.08s	remaining: 1m 59s
9:	learn: 0.6519972	total: 1.2s	remaining: 1m 59s
10:	learn: 0.6506048	total: 1.32s	remaining: 1m 59s
11:	learn: 0.6481721	total: 1.44s	remaining: 1m 58s
12:	learn: 0.6462651	total: 1.56s	remaining: 1m 58s
13:	learn: 0.6444180	total: 1.72s	remaining: 2m 1s
14:	learn: 0.6431614	total: 1.83s	remaining: 2m
15:	learn: 0.6408437	total: 1.95s	remaining: 2m
16:	learn: 0.6394367	total: 2.07s	remaining: 1m 59s
17:	learn: 0.6381160	total: 2.19s	remaining: 1m 59s
18:	learn: 0.6366392	total: 2.31s	remaining: 1m 59s
19:	learn: 0.6354083	total: 2.43s	remaini

## SKLstack

In [ ]:
# generate_predictions(
# 	"SKLstack1"
# )

# cv_grid_search("SKLstack1", grid={
# 	"penalty": ["l2"],
# 	"C": [1.0]
# }, k=5)

cv_f1s = {}
for stack_name in ["SKLstack1", "SKLstack2", "SKLstack3", "SKLstack4"]:
	model_class, default_params = MODEL_TO_CLASS_TO_DEFAULT_PARAMETERS[stack_name]
	model = model_class(**default_params)
	f1s = cross_val_score(model, X_train, y_train, cv=5, scoring="f1")
	cv_f1s[stack_name] = f1s.mean()

best_stack = max(cv_f1s, key=cv_f1s.get)
best_f1 = cv_f1s[best_stack]
print(f"Best stack: {best_stack}")
print(f"Mean cross-validated f1 for best stack: {best_f1:.4f}")

# Appendix
- Models previously implemented which are now scrapped include SKLknn, SKLsvm, SKLrf, and SKLsgd. May they rest in peace.